Stark Lab enhancers (https://enhancers.starklab.org/) dataset contains only arounf 3500 positive samples. And I would also have to generate negative samples, but I still don't have a good way of doing that.

In [4]:
%load_ext autoreload
%autoreload 2

In [26]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

# Prepare directories

In [6]:
BASE_FILE_PATH = Path("../../datasets/drosophila_enhancers_stark/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)
    
BASE_FILE_PATH.mkdir()
(BASE_FILE_PATH / 'train').mkdir()
(BASE_FILE_PATH / 'test').mkdir()

In [3]:
!wget https://enhancers.starklab.org/download/tiles/all -O tiles.csv

--2022-06-21 10:06:41--  https://enhancers.starklab.org/download/tiles/all
Resolving enhancers.starklab.org (enhancers.starklab.org)... 193.171.188.109
Connecting to enhancers.starklab.org (enhancers.starklab.org)|193.171.188.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957907 (935K) [text/csv]
Saving to: ‘tiles.csv’

tiles.csv           100%[===================>] 935,46K  6,05MB/s    in 0,2s    

2022-06-21 10:06:41 (6,05 MB/s) - ‘tiles.csv’ saved [957907/957907]



In [21]:
df = pd.read_csv('tiles.csv')
df

,VTID,Chrosome,Start,End,Length,Verification Status,Positive,stg4_6,stg7_8,stg9_10,stg11_12,stg13_14,stg15_16
0,VT0002,chr2L,9410,10052,642,correct,0,NaN,NaN,NaN,NaN,NaN,NaN
1,VT0003,chr2L,11245,11850,605,correct,1,NaN,NaN,NaN,NaN,NaN,hindgut;2|posterior_hindgut;3
2,VT0004,chr2L,12730,13724,994,correct,0,NaN,NaN,NaN,NaN,NaN,NaN
3,VT0005,chr2L,15507,17836,2329,correct,0,NaN,NaN,NaN,NaN,dorsal_epidermis_subset;1,dorsal_epidermis_subset;1
4,VT0006,chr2L,16836,18924,2088,correct,1,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;3,brain_broad;3|ubiquitous;3|ventral_nerve_cord_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7788,VT9951,chr2L,19419622,19421726,2104,correct,1,NaN,NaN,NaN,NaN,lateral_epidermis_subset;3|ventral_epidermis_s...,head_epidermis_ventral_subset;3|lateral_epider...
7789,VT9978,chr2L,19469452,19471821,2369,correct,1,NaN,NaN,NaN,NaN,ventral_nerve_cord_subset;2|brain_subset;2,ventral_nerve_cord_subset;2|brain_subset;2
7790,VT9983,chr2L,19477513,19479654,2141,correct,1,NaN,NaN,NaN,NaN,ventral_nerve_cord_subset;1|brain_subset;1,ventral_nerve_cord_subset;1|midgut_chamber;2|b...
7791,VT9998,chr2L,19510549,19511569,1020,correct,0,NaN,NaN,NaN,NaN,NaN,NaN


## Verification status

From https://enhancers.starklab.org/methods: "The identity of 7478 lines (96%) was confirmed using PCR of genomic DNA isolated from transgenic flies followed by Sanger sequencing (the remaining 4% were either not tested or failed)."

In [11]:
df['Verification Status'].value_counts()

correct         7478
not_verified     315
Name: Verification Status, dtype: int64

## Positive

From https://www.nature.com/articles/nature13395: We also manually annotated the strength of the signal for each annotation term semiquantitatively from 1 (very weak) to 5 (very strong). A line was considered as positive if it contained at least one annotation term with strength >1 (excluding enhancers with only very weak (1) activities). This resulted in 3,557 positively annotated VT strains (4,480 (58.1%) if also considering very weak signals).


In [12]:
df['Positive'].value_counts()

0    4189
1    3604
Name: Positive, dtype: int64

In [24]:
df = df[(df['Verification Status'] == 'correct') & (df['Positive'] == 1)]
df = df.reset_index()
df

,index,VTID,Chrosome,Start,End,Length,Verification Status,Positive,stg4_6,stg7_8,stg9_10,stg11_12,stg13_14,stg15_16
0,1,VT0003,chr2L,11245,11850,605,correct,1,NaN,NaN,NaN,NaN,NaN,hindgut;2|posterior_hindgut;3
1,4,VT0006,chr2L,16836,18924,2088,correct,1,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;3,brain_broad;3|ubiquitous;3|ventral_nerve_cord_...
2,5,VT0007,chr2L,18031,20238,2207,correct,1,ubiquitous;3,ubiquitous;4,ubiquitous;4,ubiquitous;4,ubiquitous;4,midgut_chamber;4|ubiquitous;4
3,10,VT0013,chr2L,26520,27246,726,correct,1,procephalic_ectoderm_AISN;4,procephalic_ectoderm_anlage;3|head_mesoderm_an...,NaN,NaN,NaN,NaN
4,19,VT0025,chr2L,54047,56104,2057,correct,1,NaN,NaN,NaN,NaN,NaN,antenno-maxillary-labial_complex;4|ventral_epi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3452,7784,VT9946,chr2L,19410749,19412957,2208,correct,1,NaN,NaN,NaN,amnioserosa;2,amnioserosa;3,amnioserosa;3
3453,7786,VT9949,chr2L,19415569,19417675,2106,correct,1,NaN,NaN,NaN,hindgut_proper_primordium;1|posterior_midgut_p...,hindgut;2|midgut_subset;2,head_subset;3|midgut_broad;3|hindgut;3|salivar...
3454,7788,VT9951,chr2L,19419622,19421726,2104,correct,1,NaN,NaN,NaN,NaN,lateral_epidermis_subset;3|ventral_epidermis_s...,head_epidermis_ventral_subset;3|lateral_epider...
3455,7789,VT9978,chr2L,19469452,19471821,2369,correct,1,NaN,NaN,NaN,NaN,ventral_nerve_cord_subset;2|brain_subset;2,ventral_nerve_cord_subset;2|brain_subset;2


In [25]:
df = df[['Chrosome', 'Start', 'End']]
df.columns = ['region', 'start', 'end']
df['strand'] = '+'
df

/tmp/ipykernel_106695/3624373002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['strand'] = '+'


,region,start,end,strand
0,chr2L,11245,11850,+
1,chr2L,16836,18924,+
2,chr2L,18031,20238,+
3,chr2L,26520,27246,+
4,chr2L,54047,56104,+
...,...,...,...,...
3452,chr2L,19410749,19412957,+
3453,chr2L,19415569,19417675,+
3454,chr2L,19419622,19421726,+
3455,chr2L,19469452,19471821,+


In [27]:
train_pos_seqs, test_pos_seqs = train_test_split(df, shuffle=True, random_state=42)
train_pos_seqs.shape, test_pos_seqs.shape

((2592, 4), (865, 4))